# Handwritten Digit Recognition Model

This Convolutional Neural Network (CNN) model is designed to recognize handwritten digits from 0-9. It's trained on the MNIST dataset and optimized for real-time prediction on an Arduino Nano project with a seven-segment display.

## Model Architecture

The model uses a modern CNN architecture with:
- Two convolutional layers with batch normalization
- Max pooling for dimensionality reduction
- Dropout regularization (50%) to prevent overfitting
- ReLU activation functions throughout the network

```
DigitCNN Architecture:
- Conv2D(1→16, 3×3) + BatchNorm2D + ReLU
- MaxPool2D(2×2)
- Conv2D(16→32, 3×3) + BatchNorm2D + ReLU
- MaxPool2D(2×2)
- Fully Connected Layer (800→128) + ReLU
- Dropout(0.5)
- Output Layer (128→10)
```

## Training Approach

The model is trained with:
- Data augmentation (rotation, translation) for better generalization
- Adam optimizer with learning rate of 0.001
- Cross-entropy loss function
- 90/10 train-validation split
- Early stopping based on validation accuracy

## Performance

After 10 epochs of training, the model achieves:
- Training loss: ~0.XX
- Validation accuracy: ~9X.X%

The model is lightweight enough to run on embedded systems while maintaining high accuracy for clear handwritten digits.

## Usage

This model is designed to be exported and used in a Python application that captures handwritten input from a canvas, processes it to match the MNIST format, and sends predictions to an Arduino Nano to display on a seven-segment display.

# THE CODE

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split

**Data augmentation and transformation**

In [ ]:
transform = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.ToTensor()
])

**Load MNIST dataset && Split into training and validation sets**

In [ ]:
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)


train_size = int(0.9 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_data, val_data = random_split(train_dataset, [train_size, val_size])
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64, shuffle=False)

**CNN model**

In [ ]:
class DigitCNN(nn.Module):
    def __init__(self):
        super(DigitCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, 1)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, 3, 1)
        self.bn2 = nn.BatchNorm2d(32)
        self.fc1 = nn.Linear(32 * 5 * 5, 128)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))  # [1, 28, 28] → [16, 26, 26]
        x = F.max_pool2d(x, 2)               # → [16, 13, 13]
        x = F.relu(self.bn2(self.conv2(x)))  # → [32, 11, 11]
        x = F.max_pool2d(x, 2)               # → [32, 5, 5]
        x = x.view(-1, 32 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


**Initialize model, optimizer, and loss**

In [ ]:
model = DigitCNN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()


**Training loop with validation**

In [ ]:
for epoch in range(10):
    model.train()
    train_loss = 0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    # Validation
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            val_loss += loss_fn(outputs, labels).item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Epoch {epoch+1}, Train Loss: {train_loss/len(train_loader):.4f}, "
          f"Val Loss: {val_loss/len(val_loader):.4f}, Val Acc: {100 * correct/total:.2f}%")

**Save the model**

In [ ]:
torch.save(model.state_dict(), "digit_cnn.pth")